# Greedy Algorithm on MaxCut Problem

In this notebook, we introduce Greedy Algorithm implementation for the MaxCut Problem.

1. [Introduction](#intro)
2. [Greedy Function](#greedy-function)
3. [Testing](#test)
4. [Benchmarked Results](#results)

<a id='intro'></a>
## 1. Introduction

### Description

A greedy algorithm constructs a solution by selecting the best option at each step with the hope that this local optimization leads to a globally optimal solution. It is efficient for problems with the greedy-choice property and optimal substructure but doesn't guarantee the optimal solution for all problem types.

## Imports

In [ ]:
%pip install numpy
%pip install networkx
%pip install torch
%pip install pandas

In [1]:
import copy
import time
from typing import List, Union
import numpy as np
import networkx as nx

from util import read_nxgraph
from util import obj_maxcut
from util import transfer_nxgraph_to_weightmatrix

copy: deep copies <br>
time: measurement <br>
typing: type annotations <br>
numpy: arrays<br>
networkx: graph manipulation/analysis <br>
util: read graphs, calculate Max-Cut objective function, convert graph to weight matrix

<a id='greedy-function'></a>
## 2. Greedy Function

In [2]:
def greedy_maxcut(init_solution, num_steps: int, graph: nx.Graph) -> (int, Union[List[int], np.array], List[int]):
    """
    Performs greedy algorithm for Max Cut.

    Parameters:
    -- init_solution: Initial solution
    -- num_steps: Number of steps
    -- graph: A NetworkX graph for the MaxCut problem

    Return:
    -- curr_score: Final objective value after greedy algorithm
    -- curr_solution: Final solution partitioning of nodes
    -- scores: List of objective values at each step
    """
    # function initialization
    print('greedy')
    start_time = time.time()
    num_nodes = int(graph.number_of_nodes())
    nodes = list(range(num_nodes))
    assert sum(init_solution) == 0
    if num_steps is None:
        num_steps = num_nodes
    curr_solution = copy.deepcopy(init_solution)
    curr_score: int = obj_maxcut(curr_solution, graph)
    init_score = curr_score
    scores = []
    # loop
    for iteration in range(num_nodes):
        if iteration >= num_steps:
            break
        score = obj_maxcut(curr_solution, graph)
        print(f"iteration: {iteration}, score: {score}")
        traversal_scores = []
        traversal_solutions = []
        # nested loop
        # calc the new solution when moving to a new node. Then store the scores and solutions.
        for node in nodes:
            new_solution = copy.deepcopy(curr_solution)
            # search a new solution and calc obj
            new_solution[node] = (new_solution[node] + 1) % 2
            new_score = obj_maxcut(new_solution, graph)
            traversal_scores.append(new_score)
            traversal_solutions.append(new_solution)
        # save best score
        best_score = max(traversal_scores)
        index = traversal_scores.index(best_score)
        best_solution = traversal_solutions[index]
        if best_score > curr_score:
            scores.append(best_score)
            curr_score = best_score
            curr_solution = best_solution
        else:
            break
    # prints
    print("score, init_score of greedy:", curr_score, init_score)
    print("scores: ", traversal_scores)
    print("solution: ", curr_solution)
    running_duration = time.time() - start_time
    print('running_duration: ', running_duration)
    return curr_score, curr_solution, scores

<a id="test"></a>
## 3. Testing
Below are descriptions of the graphs in our dataset (sourced from https://web.stanford.edu/~yyye/yyye/Gset/). 
| Graph | # Nodes | # Edges |
|-------|---------|---------|
|  G14  |   800   |   4694  |
|  G15  |   800   |   4661  |
|  G22  |  2000   |  19990  |
|  G49  |  3000   |   6000  |
|  G50  |  3000   |   6000  |
|  G55  |  5000   |  12468  |
|  G70  | 10000   |   9999  |

Run the following to get the results of greedy algorithm on G14.

In [5]:
if __name__ == '__main__':
    graph = read_nxgraph('data/gset_14.txt')
    init_solution = [0] * graph.number_of_nodes()
    ga_score, ga_solution, ga_scores = greedy_maxcut(init_solution=init_solution, num_steps=30, graph=graph)

greedy
iteration: 0, score: 0
iteration: 1, score: 132.0
iteration: 2, score: 241.0
iteration: 3, score: 337.0
iteration: 4, score: 427.0
iteration: 5, score: 511.0
iteration: 6, score: 583.0
iteration: 7, score: 652.0
iteration: 8, score: 721.0
iteration: 9, score: 784.0
iteration: 10, score: 836.0
iteration: 11, score: 884.0
iteration: 12, score: 928.0
iteration: 13, score: 971.0
iteration: 14, score: 1012.0
iteration: 15, score: 1050.0
iteration: 16, score: 1085.0
iteration: 17, score: 1120.0
iteration: 18, score: 1155.0
iteration: 19, score: 1186.0
iteration: 20, score: 1216.0
iteration: 21, score: 1245.0
iteration: 22, score: 1274.0
iteration: 23, score: 1303.0
iteration: 24, score: 1331.0
iteration: 25, score: 1359.0
iteration: 26, score: 1387.0
iteration: 27, score: 1415.0
iteration: 28, score: 1442.0
iteration: 29, score: 1468.0
score, init_score of greedy: 1492.0 0
scores:  [1400.0, 1478.0, 1400.0, 1366.0, 1404.0, 1426.0, 1387.0, 1431.0, 1407.0, 1440.0, 1489.0, 1424.0, 1484.0,

<a id='results'></a>
## 4. Benchmarked Results

Due to the inefficiency of the greedy algorithm, the normal 7 Gset graphs we were using to benchmark all MaxCut algorithms were not feasible. Therefore, we only opted to provide data for two of them: Gset 14 and Gset 15

![title](images/greedy_scores.png)